In [1]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from nltk import tokenize
from konlpy.tag import Okt
%matplotlib inline
import matplotlib
import matplotlib.font_manager as fm
from sklearn.decomposition import PCA
import nltk

In [9]:
samsung_article = pd.read_csv('./삼성전자기사분봉.csv')
samsung_article.head(3)

,datetime,open,high,low,close,prev,article
0,2018-10-29 09:20,40800,40900,40750,40850,100.0,【서울뉴시스】이진영 기자 반도체 투톱 삼성전자와 SK하이닉스가 29일 장 초반 약...
1,2018-10-29 09:56,41050,41100,41050,41100,50.0,용인문화재단이사장 백군기이 15일부터 19일까지 삼성전자 화성캠퍼스 DSR타워 로비...
2,2018-10-29 14:26,41850,41850,41750,41750,-100.0,‘대륙의 완벽한 실수’ ‘갓성비 스마트폰’이라는 별명이 붙은 중국 샤오미의 새 ...


In [5]:
samsung_article.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1215 entries, 0 to 1214
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   datetime  1215 non-null   object
 1   article   1215 non-null   object
dtypes: object(2)
memory usage: 19.1+ KB


In [12]:
ss_datetime =samsung_article['datetime'].str.replace('.','')

In [14]:
ss_datetime.str.replace(':','')

0       202011131652
1       202011131415
2       202011101616
3       202011072005
4       202011072001
            ...     
1210    201810280937
1211    201810280937
1212    201810280937
1213    201810280937
1214    201810280917
Name: datetime, Length: 1215, dtype: object

In [10]:
samsung_bunbong = pd.read_csv('./삼성전자분봉.csv')
samsung_bunbong.head(3)

,date,time,open,high,low,close,prev,volume,tr_amount,sales_qu,purchase_qu
0,20181031,901,42900,43100,42850,43000,0,889398,38175620000,106987,781820
1,20181031,902,43000,43150,42950,43050,0,243219,10467880000,196011,936015
2,20181031,903,43050,43250,43000,43250,0,181695,7845530000,273021,1040700


In [4]:
samsung_bunbong.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570579 entries, 0 to 570578
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   date         570579 non-null  int64
 1   time         570579 non-null  int64
 2   open         570579 non-null  int64
 3   high         570579 non-null  int64
 4   low          570579 non-null  int64
 5   close        570579 non-null  int64
 6   prev         570579 non-null  int64
 7   volume       570579 non-null  int64
 8   tr_amount    570579 non-null  int64
 9   sales_qu     570579 non-null  int64
 10  purchase_qu  570579 non-null  int64
dtypes: int64(11)
memory usage: 47.9 MB


In [15]:
ss_date = samsung_bunbong['date']

In [16]:
ss_time = samsung_bunbong['time']

In [29]:
samsung_bunbong['datetime']=samsung_bunbong[['date','time']].agg(''.join,axis=1)

TypeError: sequence item 0: expected str instance, int found

In [25]:
samsung_bunbong['datetime']

0         20181932
1         20181933
2         20181934
3         20181935
4         20181936
            ...   
570574    20181935
570575    20181934
570576    20181933
570577    20181932
570578    20181931
Name: datetime, Length: 570579, dtype: int64

In [3]:
ss = pd.read_csv('./삼성전자기사분봉.csv',index_col='datetime')
ss.head()

,open,high,low,close,prev,article
datetime,,,,,,
2018-10-29 09:20,40800,40900,40750,40850,100.0,【서울뉴시스】이진영 기자 반도체 투톱 삼성전자와 SK하이닉스가 29일 장 초반 약...
2018-10-29 09:56,41050,41100,41050,41100,50.0,용인문화재단이사장 백군기이 15일부터 19일까지 삼성전자 화성캠퍼스 DSR타워 로비...
2018-10-29 14:26,41850,41850,41750,41750,-100.0,‘대륙의 완벽한 실수’ ‘갓성비 스마트폰’이라는 별명이 붙은 중국 샤오미의 새 ...
2018-10-29 15:30,41400,41400,41400,41400,-100.0,기술의 변화와 혁신이 전례 없는 규모와 속도로 이뤄지는 대전환의 시대를 맞고 있습니...
2018-10-30 09:30,42450,42450,42400,42400,-50.0,【서울뉴시스】 김정호 기자 코스피 시가총액 1 2위 종목인 삼성전자005930와 ...


In [4]:
okt = Okt()

In [5]:
# 불용어 처리
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','을','으로','자','에','와','한','하다',",",'"','`','.','에'
            ,'된다','(',')','이다','중','없다','인','것','보인다','밝혔다','것','하는','받았다','바','며']

In [6]:
ss_dataset = [] # 필터링용
top30_content = [] # top30
top30_vector =[]
for a in ss['article']:
    ss_nouns = okt.nouns(a) # 명사만 추출
    for n in ss_nouns:
        if n.isalpha(): # 문자 판별
            ss_dataset.append(n)
        else:
            continue
    for w in ss_dataset: # 2글자 판별
        if len(w) ==1:
            ss_dataset.remove(w)
        else:
            continue
    # 불용어 처리
    ss_stopwords = [s for s in ss_dataset if n not in stopwords]
    
    # Text화
    ss_text = nltk.Text(ss_stopwords)
    
    ss_vocab = ss_text.vocab()
    
    top30 = ss_vocab.most_common(30)
    
    for t in top30:
        line =[]
        for j in range(1):
            line.append(t[0])
        top30_content.append(line)
    
    # word2vec 모델 생성
    model = Word2Vec(
        top30_content
        ,size = 200
        ,window =5
        ,min_count=1
        ,workers=1
        ,sg=0
    )
    
    # 단어 벡터를 구한다
    word_vectors = model.wv
    vocabs = word_vectors.vocab.keys()
    word_vectors_list = [word_vectors[v] for v in vocabs]
    
    # 차원 낮추기
    pca = PCA(n_components=1)
    xys = pca.fit_transform(word_vectors_list)
    
    for x in xys:
        top30_vector.append(list(x))
        
article_vector = pd.Series(top30_vector)
article_vector    
    

0           [-0.004030131413137161, -0.00114170706838422]
1           [0.0018293926595306733, -0.00470456535466528]
2          [0.0030788626273105984, -0.005132323333108335]
3          [-0.005528615332526903, -0.006209602467621366]
4           [-0.0018473146326840899, 0.00650579128581048]
                               ...                       
85290    [-0.0003820735027753956, -0.0003089920260410931]
85291     [-0.001037990972061065, -0.0009559564009036167]
85292      [0.001825467699910973, -0.0008396786044766381]
85293       [0.0021433908844503834, 0.009480662969260182]
85294      [0.0005482683423023943, -0.004164491452231754]
Length: 85295, dtype: object